# 1. Load the libraries

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import cv2
import glob
import numpy as np
import pandas as pd  
from PIL import Image
from skimage import measure
from keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import tensorflow as tf
from skimage.measure import label, regionprops, regionprops_table
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, AveragePooling2D
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input


2024-11-20 20:00:50.793814: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-20 20:00:51.737163: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 2. EfficientNetB0

## 2.1 Read images and compute 99% quantile

In [2]:
#define the file paths for the images and corresponding masks
image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'
mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'

#use the glob library to generate lists of image and mask filenames
images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

ch1 = [] #fibroblast
ch2 = [] #cancer
ch4 = [] #dapi

for i in range(0, 1152, 3):
    
    CH1_path = os.path.join(image_path, images[i])
    CH2_path = os.path.join(image_path, images[i+2]) #for MHB ch3 is cancer 
    CH4_path = os.path.join(image_path, images[i+1]) #for MHB ch2 is DAPI

    #read images and masks using the Image library, and converts them to numpy arrays.
    ch1_img = Image.open(CH1_path)
    ch2_img = Image.open(CH2_path)
    ch4_img = Image.open(CH4_path)

    ch1.append(np.array(ch1_img))
    ch2.append(np.array(ch2_img))
    ch4.append(np.array(ch4_img))

ch1_max = np.max(ch1)
ch2_max = np.max(ch2)
ch4_max = np.max(ch4)
print("Maximum intensity for channel 1:", ch1_max)
print("Maximum intensity for channel 2:", ch2_max)
print("Maximum intensity for channel 4:", ch4_max)

ch1_q099 =  np.quantile(ch1, 0.99)
ch2_q099 =  np.quantile(ch2, 0.99)
ch4_q099 =  np.quantile(ch4, 0.99)
print("Quantile_099 channel 1:", ch1_q099)
print("Quantile_099 channel 2:", ch2_q099)
print("Quantile_099 channel 4:", ch4_q099)

del ch1, ch2, ch4

Maximum intensity for channel 1: 65289
Maximum intensity for channel 2: 65535
Maximum intensity for channel 4: 64458
Quantile_099 channel 1: 1959.0
Quantile_099 channel 2: 2514.0
Quantile_099 channel 4: 5185.0


## 2.2 Load the model and Read images as tensors and extract features

In [ ]:
import tensorflow as tf

# Enable memory growth for GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Logical devices:", tf.config.list_logical_devices())
        
with tf.device('/CPU:0'):
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=(90, 90, 3))
    layer = base_model.get_layer(name="top_conv").output
    output = GlobalAveragePooling2D()(layer)
    feature_extraction_model = Model(inputs=base_model.inputs, outputs=output)

print("Model loaded successfully.")

# Use MirroredStrategy to automatically distribute across GPUs 0 and 1
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# All the code inside this scope will be distributed across both GPUs
with strategy.scope():

    @tf.function
    # Example of preprocessing and extracting features
    def preprocess_image(image):
        image = tf.image.resize(image, (90, 90))  # Resize to match model input
        image = preprocess_input(image)  # Apply EfficientNet-specific preprocessing
        return image
    @tf.function
    def extract_features(image):
        preprocessed_image = preprocess_image(image)
        features = feature_extraction_model(preprocessed_image)
        return features

    #define the file paths for the images and corresponding masks
    image_path = '/4tbint/Corrected Merged FOVs/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'
    mask_path = '/4tbint/Cellpose Masks/Set5/20200408_N7863__2020-04-08T10_57_43-Measurement 2'

    #use the glob library to generate lists of image and mask filenames
    images =  sorted([os.path.basename(x) for x in glob.glob(image_path + '/*.tiff')])
    cellpose_mask = sorted([os.path.basename(x) for x in glob.glob(mask_path + '/*.tiff')])

    # "mean_feature_list" and "j" variables are initialized to empty list and 0, respectively to store the features for each image and to iterate over the list of masks.
    mean_fetaure_list =[]
    image_list = []
    j=0

    #loop with step size of 3 to iterate over a range of indices, pulling the filenames for three channels of each image.
    for i in range(0, 1152, 3):

        CH1_path = os.path.join(image_path, images[i])
        CH2_path = os.path.join(image_path, images[i+2])
        CH4_path = os.path.join(image_path, images[i+1])

        #read images and masks using the Image library, and converts them to numpy arrays.
        ch1_img = Image.open(CH1_path)
        ch2_img = Image.open(CH2_path)
        ch4_img = Image.open(CH4_path)

        image_array_ch1 = np.array(ch1_img)
        image_array_ch2 = np.array(ch2_img)
        image_array_ch4 = np.array(ch4_img)

        #use cellpose mask to extract individual cell regions,to generate 50x50 pixel images.
        cellpose_path = os.path.join(mask_path, cellpose_mask[j])
        cellpose_img = Image.open(cellpose_path)
        masks_ch2 = np.array(cellpose_img)

        labels_ch2 = label(masks_ch2)
        props = regionprops(labels_ch2)

        j=j+1
        single_cell_bbox = []

        for cell_label, target_region in enumerate(props, start=1):
            centroid_row, centroid_col = target_region.centroid

            half_size = 45
            min_row = int(max(centroid_row - half_size, 0))
            max_row = int(min(centroid_row + half_size, image_array_ch2.shape[0]))
            min_col = int(max(centroid_col - half_size, 0))
            max_col = int(min(centroid_col + half_size, image_array_ch2.shape[1]))

            if (min_row > 0 and max_row < image_array_ch2.shape[0] and 
            min_col > 0 and max_col < image_array_ch2.shape[1]):

                target_cell_mask = labels_ch2 == cell_label

                cell_area_original = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch2[min_row:max_row, min_col:max_col], 0)

                empty_array_ch2 = np.zeros((90, 90))
                paste_row_original = (90 - cell_area_original.shape[0]) // 2
                paste_col_original = (90 - cell_area_original.shape[1]) // 2

                empty_array_ch2[paste_row_original:paste_row_original+cell_area_original.shape[0], paste_col_original:paste_col_original+cell_area_original.shape[1]] = cell_area_original

                empty_array_cellpose = np.zeros((90, 90))
                cell_area_cellpose = target_cell_mask[min_row:max_row, min_col:max_col]

                paste_row_cellpose = (90 - cell_area_cellpose.shape[0]) // 2
                paste_col_cellpose = (90 - cell_area_cellpose.shape[1]) // 2

                empty_array_cellpose[paste_row_cellpose:paste_row_cellpose+cell_area_cellpose.shape[0], paste_col_cellpose:paste_col_cellpose+cell_area_cellpose.shape[1]] = cell_area_cellpose

                cell_area_ch1 = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch1[min_row:max_row, min_col:max_col], 0)
                cell_area_ch4 = np.where(target_cell_mask[min_row:max_row, min_col:max_col], image_array_ch4[min_row:max_row, min_col:max_col], 0)

                empty_array_ch1 = np.zeros((90, 90))
                paste_row_ch1 = (90 - cell_area_ch1.shape[0]) // 2
                paste_col_ch1 = (90 - cell_area_ch1.shape[1]) // 2

                empty_array_ch4 = np.zeros((90, 90))
                paste_row_ch4 = (90 - cell_area_ch4.shape[0]) // 2
                paste_col_ch4 = (90 - cell_area_ch4.shape[1]) // 2

                empty_array_ch1[paste_row_ch1:paste_row_ch1+cell_area_ch1.shape[0], paste_col_ch1:paste_col_ch1+cell_area_ch1.shape[1]] = cell_area_ch1
                empty_array_ch4[paste_row_ch4:paste_row_ch4+cell_area_ch4.shape[0], paste_col_ch4:paste_col_ch4+cell_area_ch4.shape[1]] = cell_area_ch4

                empty_array_ch1 = (empty_array_ch1.astype('float32')/ch1_q099)*255
                empty_array_ch2 = (empty_array_ch2.astype('float32')/ch2_q099)*255
                empty_array_ch4 = (empty_array_ch4.astype('float32')/ch4_q099)*255

                singlecell = np.stack((empty_array_ch1, empty_array_ch2, empty_array_ch4), axis=-1)
                reshaped_singlecell = singlecell.reshape((1,90,90,3))
                single_cell_bbox.append(reshaped_singlecell)
                #print(np.max(empty_array_ch1))

        single_cells = np.array(single_cell_bbox)

        if len(single_cells) != 0:

            print(images[i+2])

            #convert single cell objects into a tensor using tf.data.Dataset.from_tensor_slices.
            single_cell_tensor = tf.data.Dataset.from_tensor_slices(single_cells)

            #A map operation is used to apply a feature extraction function, extract_features, to each cell image 
            #using a pre-trained model. This creates a feature_dataset of extracted features.
            feature_dataset = single_cell_tensor.map(extract_features)

            #features are summed across all cells, divided by the number of cells, and added to the mean_feature_list.
            sum_feature_dataset = tf.zeros((1,1280))

            for f in feature_dataset:
                sum_feature_dataset += f

            mean_feature = sum_feature_dataset/len(feature_dataset)

            mean_fetaure_list.append(mean_feature)
            image_list.append(images[i])
        tf.keras.backend.clear_session()


Num GPUs Available:  1
Logical devices: [LogicalDevice(name='/device:CPU:0', device_type='CPU'), LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2024-11-20 20:03:33.553518: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9612 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5


Model loaded successfully.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
r01c01ch3.tiff


/home/gauss/miniconda3/envs/tf24/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
2024-11-20 20:04:10.368372: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r01c02ch3.tiff


...

r14c07ch3.tiff


2024-11-20 23:46:32.748411: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r14c08ch3.tiff


2024-11-20 23:47:15.273564: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


r14c09ch3.tiff


In [4]:
#Convert mean feature tensor array to numpy array
feature_list = [tensor.numpy() for tensor in mean_fetaure_list]

In [5]:
print(len(feature_list))
print(len(image_list))

382
382


In [6]:
# Create a 2D numpy array from the feature list
feature_array = np.vstack(feature_list)

# Create a DataFrame with column names
df = pd.DataFrame(feature_array, columns=['feature{}'.format(i+1) for i in range(1280)])

#add col to the beginning of the dataframe
df.insert(0, 'Image', image_list)
df

,Image,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature1271,feature1272,feature1273,feature1274,feature1275,feature1276,feature1277,feature1278,feature1279,feature1280
0,r01c01ch1.tiff,0.352117,0.282027,0.823517,2.094099,1.687996,0.266001,-1.078931,1.133552,0.378835,...,-1.848236,2.937428,1.009241,0.504480,-2.005462,0.931984,-0.021889,0.516383,0.238684,1.800121
1,r01c02ch1.tiff,-0.942438,0.466055,0.524793,0.779925,0.578955,1.523881,-1.251297,-0.815562,1.643063,...,-1.813078,2.713460,2.220799,1.066618,-1.868204,0.378131,0.025882,2.025434,-0.859762,0.191245
2,r01c03ch1.tiff,0.618095,-0.327209,0.383259,2.467492,1.183713,1.153427,-0.199820,0.509336,1.422256,...,-0.949752,3.963709,0.255307,1.122296,-2.373588,0.687005,-0.989506,0.795153,0.971466,2.784611
3,r01c04ch1.tiff,-0.890213,0.693963,0.044631,1.513158,0.639226,2.242331,-1.091786,-0.715966,1.693623,...,-1.881914,2.634981,2.054865,0.844694,-1.375218,0.796723,-0.161537,1.796215,-0.269378,1.034553
4,r01c05ch1.tiff,0.229088,0.706598,1.713097,1.633374,1.146331,1.644056,-1.034579,0.144617,1.476742,...,-1.134224,2.136149,1.928163,0.376727,-0.716631,1.505097,-0.066372,1.167267,0.205478,3.004736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,r16c19ch1.tiff,-0.663177,0.080941,0.233102,2.711483,0.998242,1.232759,-0.673327,0.704331,1.475150,...,-0.890875,4.250765,1.338629,1.454537,-0.760302,0.659848,1.129429,1.649723,0.600739,2.750279
378,r16c20ch1.tiff,-0.215960,-0.027819,0.376417,1.979818,0.859278,1.323526,-0.836211,0.096750,1.902651,...,-0.921419,3.222526,1.266361,0.800258,-1.558151,0.616491,0.057327,1.436998,0.383313,2.558857
379,r16c21ch1.tiff,0.375982,0.245211,0.716538,2.154788,1.674600,0.472073,-0.843587,1.033263,0.525325,...,-1.707087,3.009067,0.868366,0.543335,-2.206285,0.939899,-0.292973,0.398798,0.286558,1.848366
380,r16c22ch1.tiff,0.415212,0.249344,0.785801,2.125556,1.738448,0.353707,-0.899041,1.085130,0.434355,...,-1.766105,2.974287,0.881019,0.540056,-2.195512,0.979070,-0.242401,0.425364,0.231371,1.852793


In [7]:
#save dataframe as excel 
df.to_excel("/home/gauss/Desktop/Revised_Data_Features/Pretrained baseline EfficientNetB0 Bbox Size 90/MHB/Plate_7.xlsx")


In [ ]:
len(image_list)

In [ ]:
image_list[0:150]